In [1]:
import requests, json
from pprint import pprint
import pandas as pd
import numpy as np

In [17]:
# Build data source statistics
data_sources = []

In [18]:
file_name = "2016-17_merged_gw.csv"
df_1617 = pd.read_csv(file_name, header = 0,delimiter = ",",encoding = "ISO-8859-1")
file_summary = [file_name, len(df_1617),len(df_1617.columns),",".join(col for col in df_1617)]
data_sources.append(file_summary)

In [19]:
data_sources

[['2016-17_merged_gw.csv',
  23679,
  56,
  'name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW']]

### EDA Group by Week and count player

In [ ]:
# function to aggregate play stats over the season
def agg_player(x):
        d = []
        d.append(x['GW'].nunique())
        d.append(x['total_points'].max())
        d.append(x['total_points'].min())
        d.append(x['total_points'].mean())
        d.append(x['total_points'].sum())
        d.append(x['minutes'].mean())
        d.append(x['minutes'].sum())
        d.append(x['value'].mean())
        return pd.Series(d, index= ['GW_participated', 'total_points_max','total_points_min','total_points_mean','total_points_sum','minutes_mean','minutes_sum','value_mean'])

In [ ]:
# EDA Group by player and count week
# Get data subset: https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/03_subset_data.html
print (df_1617['element'].nunique())
df_1617_p = df_1617.reset_index()[['name', 'element','bonus', 'loaned_in', 'loaned_out','minutes','opponent_team', 'total_points', 'transfers_balance', 'transfers_in', 'transfers_out','value', 'GW']]
df_1617_p_byname = df_1617_p.groupby(['element','name']).apply(agg_player).reset_index()

In [ ]:
a[a['element'] == 77]
a['minutes_sum'].describe()

In [ ]:
### FPL API only contains detailed data for the current season and aggregate data for past seasons, so no use
### Reference: https://medium.com/analytics-vidhya/getting-started-with-fantasy-premier-league-data-56d3b9be8c32
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

# get player data from 'elements' field
players = r['elements']
events = r['events']
# show data for first player
pprint(events)

base_url = "https://fantasy.premierleague.com/api/"
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(base_url + 'element-summary/' + str(player_id) + '/').json()
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    return df

# show player #1's gameweek history
get_season_history(1)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)